# Solving Frozen Lake Problem Using Policy Iteration

Imagine, there is a frozen lake from your home to office, you should walk on the frozen lake
to reach your office. But oops! there will be a hole in the frozen lake in between, so you have
to be careful while walking in the frozen lake to avoid getting trapped at holes.
Look at the below figure where, 

1. S is the starting position (Home)
2. F is the Frozen lake where you can walk
3. H is the Hole which you have to be so careful about
4. G is the Goal (office)


![title](images/B09792_03_50.png)

Okay, now let us use our agent instead of you to find the correct way to reach the office.
The agent goal is to find the optimal path to reach from S to G without getting trapped at H.
How an agent can achieve this? We give +1 point as a reward to the agent if it correctly
walks on the frozen lake and 0 points if it falls into the hole. So that agent could determine
which is the right action. An agent will now try to find the optimal policy. Optimal policy
implies taking the correct path which maximizes the agent reward. If the agent is
maximizing the reward, apparently agent is learning to skip the hole and reach the
destination.

 First, we import necessary libraries

In [1]:
import gym
import numpy as np

Initialize our gym environment

In [2]:
env = gym.make('FrozenLake-v0')

/Users/453543/anaconda3/lib/python3.11/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment FrozenLake-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


DeprecatedEnv: Environment version v0 for `FrozenLake` is deprecated. Please use `FrozenLake-v1` instead.

Let us see how the environment looks like

In [ ]:
env.render()

SFFF
FHFH
FFFH
HFFG



Now, we will compute the value function

In [ ]:
def compute_value_function(policy, gamma=1.0):
    
    # initialize value table with zeros
    value_table = np.zeros(env.nS)
    
    # set the threshold
    threshold = 1e-10
    
    while True:
        
        # copy the value table to the updated_value_table
        updated_value_table = np.copy(value_table)

        # for each state in the environment, select the action according to the policy and compute the value table
        for state in range(env.nS):
            action = policy[state]
            
            # build the value table with the selected action
            value_table[state] = sum([trans_prob * (reward_prob + gamma * updated_value_table[next_state]) 
                        for trans_prob, next_state, reward_prob, _ in env.P[state][action]])
            
        if (np.sum((np.fabs(updated_value_table - value_table))) <= threshold):
            break
            
    return value_table


 Now, we define a function called extract policy for extracting optimal policy from the optimal value function. 
i.e We calculate Q value using our optimal value function and pick up
the actions which has the highest Q value for each state as the optimal policy.

In [ ]:

def extract_policy(value_table, gamma = 1.0):
 
    # Initialize the policy with zeros
    policy = np.zeros(env.observation_space.n) 
    
    
    for state in range(env.observation_space.n):
        
        # initialize the Q table for a state
        Q_table = np.zeros(env.action_space.n)
        
        # compute Q value for all ations in the state
        for action in range(env.action_space.n):
            for next_sr in env.P[state][action]: 
                trans_prob, next_state, reward_prob, _ = next_sr 
                Q_table[action] += (trans_prob * (reward_prob + gamma * value_table[next_state]))
        
        # Select the action which has maximum Q value as an optimal action of the state
        policy[state] = np.argmax(Q_table)
    
    return policy

Now, we define the function for performing policy iteration

In [ ]:
def policy_iteration(env,gamma = 1.0):
    
    # Initialize policy with zeros
    old_policy = np.zeros(env.observation_space.n)   
    no_of_iterations = 200000
    
    for i in range(no_of_iterations):
        
        # compute the value function
        new_value_function = compute_value_function(old_policy, gamma)
        
        # Extract new policy from the computed value function
        new_policy = extract_policy(new_value_function, gamma)
   
        # Then we check whether we have reached convergence i.e whether we found the optimal
        # policy by comparing old_policy and new policy if it same we will break the iteration
        # else we update old_policy with new_policy

        if (np.all(old_policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        old_policy = new_policy
        
    return new_policy



In [ ]:
print (policy_iteration(env))

Policy-Iteration converged at step 7.
[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]
